In [7]:
# !pip install -U liwc
# liwc pkg could be downloaded but it requires lexicon to use, which has to be purchsed.
# see: https://pypi.org/project/liwc/

# alternatively we could use EMPATH
# Guideline here: https://github.com/Ejhfast/empath-client

# If still not working: 
# Try out SEANCE. It's free, requires no programming experience, outperforms LIWC, and is open science
# https://www.linguisticanalysistools.org/seance.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for empath: filename=empath-0.89-py3-none-any.whl size=57799 sha256=43573c0f28a783353989e156b4cd6f39ee09691c2ab80cd93cb350317981726c
  Stored in directory: /Users/ychen/Library/Caches/pip/wheels/b5/93/85/f4e8694df404eb214060dd0e0f3fda14398638f5835b3dde23
Successfully built empath

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install empath
from empath import Empath
lexicon = Empath()

In [ ]:
# do an example analysis

lexicon.analyze("he kiss the other person", normalize=True)

In [18]:
# Print all category (class) names
print(list(lexicon.cats.keys()))

['help', 'office', 'dance', 'money', 'wedding', 'domestic_work', 'sleep', 'medical_emergency', 'cold', 'hate', 'cheerfulness', 'aggression', 'occupation', 'envy', 'anticipation', 'family', 'vacation', 'crime', 'attractive', 'masculine', 'prison', 'health', 'pride', 'dispute', 'nervousness', 'government', 'weakness', 'horror', 'swearing_terms', 'leisure', 'suffering', 'royalty', 'wealthy', 'tourism', 'furniture', 'school', 'magic', 'beach', 'journalism', 'morning', 'banking', 'social_media', 'exercise', 'night', 'kill', 'blue_collar_job', 'art', 'ridicule', 'play', 'computer', 'college', 'optimism', 'stealing', 'real_estate', 'home', 'divine', 'sexual', 'fear', 'irritability', 'superhero', 'business', 'driving', 'pet', 'childish', 'cooking', 'exasperation', 'religion', 'hipster', 'internet', 'surprise', 'reading', 'worship', 'leader', 'independence', 'movement', 'body', 'noise', 'eating', 'medieval', 'zest', 'confusion', 'water', 'sports', 'death', 'healing', 'legend', 'heroic', 'celebr

In [124]:
# check if a class exists
# Define the class (category) you're looking for
class_name = "swear"  

# Find matching categories (sub-string match, case insensitive)
matched_categories = [cat for cat in lexicon.cats if class_name.lower() in cat.lower()]

# Check if any categories matched and print the results
if matched_categories:
    print("YES")
    print("Matched categories:", matched_categories)
else:
    print("NO")


YES
Matched categories: ['swearing_terms']


In [126]:
# Use Yarkoni (2011) Table 1 as the benchmark

# let's see if spacy has the rest
# !pip install spacy
!python -m spacy download en_core_web_sm

import spacy
# Load the English model
nlp = spacy.load("en_core_web_sm")
# Get the list of all POS tags and their descriptions
pos_tags = nlp.vocab.pos
# Print each POS tag and its description
for pos in pos_tags:
    print(f"{pos}: {pos_tags[pos].text}")

zsh:1: command not found: python


AttributeError: 'spacy.vocab.Vocab' object has no attribute 'pos'

In [37]:

'''
Neuroticism

- Anxiety: 
    - negative_emotion / negative emotions(0.12)
    - affection / affect (0.1)
    - - 1st person (0.15)
    - - 2nd person (-0.1)
    - - articles (-0.16)
- Anger
    - negative_emotion / (0.18)
    - swearing_terms / swearing (0.14)
    - school / school (0.13)
    - - 1st person sing. (0.11) 
    - - 1st person (0.1) 
- Depression
    - Anger (0.15)
    - optimism (-0.12)
    - - 1st person pl. (-0.13) 
    - - 2nd person (-0.1) 
    - - number (-0.09)
- Self-consciousness
    - achievement (0.16)
    - friends (-0.15)
    - occupaton (0.13)
    - communication (-0.13)
    - hearing (-0.12)
    - family (-0.12)
    - religion (-0.12)
    - school (0.12)
    - - 1st person pl (-0.14)
    - - present tense VB (0.13)
    - - 1st person sing. (0.11)
    - - articles (-0.1)
 - Immoderation
     - Anger (0.18)
     - swearing_term (0.16)
     - negative_emotions (0.14)
     - optimism (-0.12)
     - - 1st person sing. (0.12)
     - - articles (-0.11)
     - - 1st person pl. (-0.11)
 - Vulnerability
     - - articles (-0.16)
     - - 1st person sing. (0.14) 
     - - 1st person (0.13)
     - - 2nd person (-0.1)

Extraversion
- Friendliness
    - firends (0.23)
    - leisure (0.22)
    - - 1st p pl. (0.22)
    - family (0.2)
    - positive_emotions (0.17)
    - sexual (0.16)
    - sports (0.15)
    - music (0,14)
    - eating (0.14)
    - optimism (0.13)
Openness
    - friends (0.26)
    - leisure (0.23)
    - sexual (0.22)
    - music (0.2)
    - positive_emotions (0.19)
    - sports (0.19)
    - - 1 p p (0.14)
    - eating (0.14)
Assertiveness
    - communication (0.14)
    - - 2 p (0.11)
    - friends (0.11)
    - - numbers (-0.1)
    - - hearing (0.16)

Agreeableness

Conscientiousness


'''

'\nNeuroticism\n- Anxiety: \n    - negative_emotion / negative emotions(0.12)\n    - affection / affect (0.1)\n- Anger\n    - negative_emotion / (0.18)\n    - swearing_terms / swearing (0.14)\n    - school / school (0.13)\n\nExtraversion\n\nOpenness\n\nAgreeableness\n\nConscientiousness\n\n\n'